# Система прогнозирования продаж и планирования отгрузок
## Wildberries & Ozon

**Цель:** Построить систему rolling-прогноза продаж на 18 месяцев с планированием отгрузок

**Период данных:** 26.03.2024 – 20.12.2025 (дневная частота)

**Идентификатор:** solo-code


In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Прогнозирование
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from prophet import Prophet

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("✅ Библиотеки загружены")


## 1. Загрузка данных

Поддерживаем загрузку CSV и Excel файлов


In [ ]:
def load_data(file_path):
    """Загрузка данных (CSV или Excel)"""
    if file_path.endswith('.csv'):
        return pd.read_csv(file_path, encoding='utf-8')
    elif file_path.endswith(('.xlsx', '.xls')):
        return pd.read_excel(file_path)
    else:
        raise ValueError(f"Неподдерживаемый формат файла: {file_path}")

# Загрузка данных продаж
try:
    wb_sales = load_data('wb_sales.csv')
    print(f"✅ WB продажи: {wb_sales.shape}")
except:
    try:
        wb_sales = load_data('wb_sales.xlsx')
        print(f"✅ WB продажи: {wb_sales.shape}")
    except:
        print("⚠️ Файл wb_sales не найден, создаем пример данных")
        dates = pd.date_range('2024-03-26', '2025-12-20', freq='D')
        wb_sales = pd.DataFrame({
            'Дата': dates,
            'solo-code': 'SKU001',
            'SKU': 'SKU001',
            'Количество упак.': np.random.poisson(10, len(dates))
        })

try:
    ozon_sales = load_data('ozon_sales.csv')
    print(f"✅ Ozon продажи: {ozon_sales.shape}")
except:
    try:
        ozon_sales = load_data('ozon_sales.xlsx')
        print(f"✅ Ozon продажи: {ozon_sales.shape}")
    except:
        print("⚠️ Файл ozon_sales не найден, создаем пример данных")
        dates = pd.date_range('2024-03-26', '2025-12-20', freq='D')
        ozon_sales = pd.DataFrame({
            'Дата': dates,
            'solo-code': 'SKU001',
            'SKU': 'SKU001',
            'Количество упак.': np.random.poisson(8, len(dates))
        })

# Загрузка остатков
try:
    wb_stocks = load_data('wb_stocks.csv')
    print(f"✅ WB остатки: {wb_stocks.shape}")
except:
    try:
        wb_stocks = load_data('wb_stocks.xlsx')
        print(f"✅ WB остатки: {wb_stocks.shape}")
    except:
        print("⚠️ Файл wb_stocks не найден, создаем пример данных")
        dates = pd.date_range('2024-03-26', '2025-12-20', freq='D')
        wb_stocks = pd.DataFrame({
            'Дата': dates,
            'Склад': 'WB',
            'solo-code': 'SKU001',
            'SKU': 'SKU001',
            'Остаток': np.random.randint(50, 200, len(dates))
        })

try:
    ozon_stocks = load_data('ozon_stocks.csv')
    print(f"✅ Ozon остатки: {ozon_stocks.shape}")
except:
    try:
        ozon_stocks = load_data('ozon_stocks.xlsx')
        print(f"✅ Ozon остатки: {ozon_stocks.shape}")
    except:
        print("⚠️ Файл ozon_stocks не найден, создаем пример данных")
        dates = pd.date_range('2024-03-26', '2025-12-20', freq='D')
        ozon_stocks = pd.DataFrame({
            'Дата': dates,
            'Склад': 'Ozon',
            'solo-code': 'SKU001',
            'SKU': 'SKU001',
            'Остаток': np.random.randint(40, 150, len(dates))
        })

try:
    our_stocks = load_data('our_stocks.csv')
    print(f"✅ Наши остатки: {our_stocks.shape}")
except:
    try:
        our_stocks = load_data('our_stocks.xlsx')
        print(f"✅ Наши остатки: {our_stocks.shape}")
    except:
        print("⚠️ Файл our_stocks не найден, создаем пример данных")
        dates = pd.date_range('2024-03-26', '2025-12-20', freq='D')
        our_stocks = pd.DataFrame({
            'Дата': dates,
            'solo-code': 'SKU001',
            'SKU': 'SKU001',
            'Остаток': np.random.randint(100, 300, len(dates))
        })

# Загрузка справочников
try:
    withdraw = load_data('withdraw.csv')
    print(f"✅ Товары на вывод: {withdraw.shape}")
except:
    try:
        withdraw = load_data('withdraw.xlsx')
        print(f"✅ Товары на вывод: {withdraw.shape}")
    except:
        print("⚠️ Файл withdraw не найден, создаем пустой")
        withdraw = pd.DataFrame(columns=['solo-code', 'SKU'])

try:
    defecture = load_data('defecture.csv')
    print(f"✅ Дефектура: {defecture.shape}")
except:
    try:
        defecture = load_data('defecture.xlsx')
        print(f"✅ Дефектура: {defecture.shape}")
    except:
        print("⚠️ Файл defecture не найден, создаем пустой")
        defecture = pd.DataFrame(columns=['solo-code', 'SKU', 'Дата окончания дефектуры'])

try:
    count_box = load_data('count_box.csv')
    print(f"✅ Количество в коробе: {count_box.shape}")
except:
    try:
        count_box = load_data('count_box.xlsx')
        print(f"✅ Количество в коробе: {count_box.shape}")
    except:
        print("⚠️ Файл count_box не найден, создаем пример")
        count_box = pd.DataFrame({
            'solo-code': ['SKU001'],
            'Кол-во': [12]
        })

print("\n📊 Обзор загруженных данных:")
print(f"WB продажи: {wb_sales.shape}")
print(f"Ozon продажи: {ozon_sales.shape}")
print(f"WB остатки: {wb_stocks.shape}")
print(f"Ozon остатки: {ozon_stocks.shape}")
print(f"Наши остатки: {our_stocks.shape}")


## 2. Подготовка и очистка данных

### 2.1. Приведение дат к datetime и создание полного календаря


In [ ]:
def prepare_dates(df, date_col='Дата'):
    """Приведение дат к datetime"""
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    return df

# Приведение дат
wb_sales = prepare_dates(wb_sales)
ozon_sales = prepare_dates(ozon_sales)
wb_stocks = prepare_dates(wb_stocks)
ozon_stocks = prepare_dates(ozon_stocks)
our_stocks = prepare_dates(our_stocks)

if 'Дата окончания дефектуры' in defecture.columns:
    defecture['Дата окончания дефектуры'] = pd.to_datetime(defecture['Дата окончания дефектуры'], errors='coerce')

# Определение общего периода
all_dates = []
for df in [wb_sales, ozon_sales, wb_stocks, ozon_stocks, our_stocks]:
    if 'Дата' in df.columns:
        all_dates.extend(df['Дата'].dropna().tolist())

min_date = min(all_dates) if all_dates else pd.Timestamp('2024-03-26')
max_date = max(all_dates) if all_dates else pd.Timestamp('2025-12-20')

print(f"📅 Период данных: {min_date.date()} - {max_date.date()}")
print(f"📅 Всего дней: {(max_date - min_date).days + 1}")

# Получение всех solo-code
all_solo_codes = set()
for df in [wb_sales, ozon_sales, wb_stocks, ozon_stocks, our_stocks]:
    if 'solo-code' in df.columns:
        all_solo_codes.update(df['solo-code'].dropna().unique())

all_solo_codes = sorted(list(all_solo_codes))
print(f"📦 Уникальных solo-code: {len(all_solo_codes)}")
print(f"📦 Примеры: {all_solo_codes[:5]}")


In [ ]:
def create_full_calendar(min_date, max_date, solo_codes):
    """Создание полного календаря для всех solo-code"""
    dates = pd.date_range(min_date, max_date, freq='D')
    calendar = []
    for solo_code in solo_codes:
        for date in dates:
            calendar.append({'Дата': date, 'solo-code': solo_code})
    return pd.DataFrame(calendar)

# Создание полного календаря
full_calendar = create_full_calendar(min_date, max_date, all_solo_codes)
print(f"📅 Полный календарь: {full_calendar.shape}")
print(f"📅 Пример календаря:")
print(full_calendar.head(10))


### 2.2. Обработка продаж: заполнение пропусков нулями


In [ ]:
def process_sales(df, calendar, channel_name):
    """Обработка продаж с заполнением пропусков"""
    # Агрегация по дате и solo-code
    sales_agg = df.groupby(['Дата', 'solo-code'])['Количество упак.'].sum().reset_index()
    
    # Объединение с календарем
    sales_full = calendar.merge(sales_agg, on=['Дата', 'solo-code'], how='left')
    sales_full['Количество упак.'] = sales_full['Количество упак.'].fillna(0)
    sales_full['Канал'] = channel_name
    
    return sales_full

# Обработка продаж
wb_sales_processed = process_sales(wb_sales, full_calendar, 'WB')
ozon_sales_processed = process_sales(ozon_sales, full_calendar, 'Ozon')

# Суммарные продажи
total_sales = wb_sales_processed.merge(
    ozon_sales_processed[['Дата', 'solo-code', 'Количество упак.']],
    on=['Дата', 'solo-code'],
    how='left',
    suffixes=('_wb', '_ozon')
)
total_sales['Количество упак.'] = (
    total_sales['Количество упак._wb'].fillna(0) + 
    total_sales['Количество упак._ozon'].fillna(0)
)
total_sales = total_sales[['Дата', 'solo-code', 'Количество упак.']].copy()
total_sales['Канал'] = 'Total'

print("✅ Продажи обработаны")
print(f"\n📊 Статистика по продажам:")
print(f"WB - пропусков: {(wb_sales_processed['Количество упак.'] == 0).sum() / len(wb_sales_processed) * 100:.1f}%")
print(f"Ozon - пропусков: {(ozon_sales_processed['Количество упак.'] == 0).sum() / len(ozon_sales_processed) * 100:.1f}%")
print(f"\n📊 Пример данных WB:")
print(wb_sales_processed.head(10))


### 2.3. Обработка остатков: forward/backward fill


In [ ]:
def process_stocks(df, calendar, stock_name):
    """Обработка остатков с forward/backward fill"""
    # Агрегация по дате и solo-code (если несколько складов - суммируем)
    if 'Склад' in df.columns:
        stocks_agg = df.groupby(['Дата', 'solo-code'])['Остаток'].sum().reset_index()
    else:
        stocks_agg = df.groupby(['Дата', 'solo-code'])['Остаток'].sum().reset_index()
    
    # Объединение с календарем
    stocks_full = calendar.merge(stocks_agg, on=['Дата', 'solo-code'], how='left')
    
    # Сортировка по solo-code и дате
    stocks_full = stocks_full.sort_values(['solo-code', 'Дата'])
    
    # Forward fill, затем backward fill для начала ряда
    stocks_full['Остаток'] = stocks_full.groupby('solo-code')['Остаток'].fillna(method='ffill')
    stocks_full['Остаток'] = stocks_full.groupby('solo-code')['Остаток'].fillna(method='bfill')
    stocks_full['Остаток'] = stocks_full['Остаток'].fillna(0)
    
    # Проверка на отрицательные значения
    stocks_full['Остаток'] = stocks_full['Остаток'].clip(lower=0)
    
    stocks_full['Тип_остатка'] = stock_name
    
    return stocks_full

# Обработка остатков
wb_stocks_processed = process_stocks(wb_stocks, full_calendar, 'WB')
ozon_stocks_processed = process_stocks(ozon_stocks, full_calendar, 'Ozon')
our_stocks_processed = process_stocks(our_stocks, full_calendar, 'Our')

print("✅ Остатки обработаны")
print(f"\n📊 Статистика по остаткам:")
print(f"WB остатки - среднее: {wb_stocks_processed['Остаток'].mean():.1f}")
print(f"Ozon остатки - среднее: {ozon_stocks_processed['Остаток'].mean():.1f}")
print(f"Наши остатки - среднее: {our_stocks_processed['Остаток'].mean():.1f}")
print(f"\n📊 Пример данных остатков:")
print(wb_stocks_processed.head(10))


### 2.4. Сводная статистика по данным


In [ ]:
print("=" * 80)
print("📊 СВОДНАЯ СТАТИСТИКА ПО ДАННЫМ")
print("=" * 80)

print(f"\n📦 Продажи:")
print(f"  - WB: {len(wb_sales_processed)} записей, {wb_sales_processed['solo-code'].nunique()} товаров")
print(f"  - Ozon: {len(ozon_sales_processed)} записей, {ozon_sales_processed['solo-code'].nunique()} товаров")
print(f"  - Total: {len(total_sales)} записей, {total_sales['solo-code'].nunique()} товаров")

print(f"\n🏬 Остатки:")
print(f"  - WB: {len(wb_stocks_processed)} записей")
print(f"  - Ozon: {len(ozon_stocks_processed)} записей")
print(f"  - Наши: {len(our_stocks_processed)} записей")

print(f"\n📈 Пропуски в продажах (нулевые дни):")
for solo_code in all_solo_codes[:5]:  # Показываем первые 5
    wb_data = wb_sales_processed[wb_sales_processed['solo-code'] == solo_code]
    ozon_data = ozon_sales_processed[ozon_sales_processed['solo-code'] == solo_code]
    total_data = total_sales[total_sales['solo-code'] == solo_code]
    
    wb_zeros = (wb_data['Количество упак.'] == 0).sum() / len(wb_data) * 100
    ozon_zeros = (ozon_data['Количество упак.'] == 0).sum() / len(ozon_data) * 100
    total_zeros = (total_data['Количество упак.'] == 0).sum() / len(total_data) * 100
    
    print(f"  {solo_code}: WB={wb_zeros:.1f}%, Ozon={ozon_zeros:.1f}%, Total={total_zeros:.1f}%")

print("\n✅ Подготовка данных завершена")


## 3. EDA и классификация спроса

Для каждого solo-code анализируем:
- График продаж
- Доля нулевых дней
- Средний спрос
- Коэффициент вариации (CV)


In [ ]:
def calculate_demand_metrics(sales_data, solo_code):
    """Расчет метрик спроса для товара"""
    data = sales_data[sales_data['solo-code'] == solo_code].copy()
    data = data.sort_values('Дата')
    
    demand = data['Количество упак.'].values
    
    metrics = {
        'solo-code': solo_code,
        'mean_demand': np.mean(demand),
        'std_demand': np.std(demand),
        'cv': np.std(demand) / (np.mean(demand) + 1e-6),  # Коэффициент вариации
        'zero_ratio': (demand == 0).sum() / len(demand),
        'total_demand': np.sum(demand),
        'max_demand': np.max(demand),
        'min_demand': np.min(demand)
    }
    
    return metrics

# Расчет метрик для всех товаров
demand_metrics = []
for solo_code in all_solo_codes:
    metrics = calculate_demand_metrics(total_sales, solo_code)
    demand_metrics.append(metrics)

demand_df = pd.DataFrame(demand_metrics)

print("📊 Метрики спроса по товарам:")
print(demand_df.head(10))
print(f"\n📊 Общая статистика:")
print(demand_df.describe())


In [ ]:
def classify_demand(zero_ratio, cv):
    """Классификация типа спроса"""
    if zero_ratio > 0.5:
        return 'intermittent'  # Прерывистый спрос (>50% нулей)
    elif cv > 1.0:
        return 'seasonal'  # Сезонный/нестабильный (высокая вариативность)
    else:
        return 'stable'  # Стабильный спрос

# Классификация товаров
demand_df['demand_type'] = demand_df.apply(
    lambda x: classify_demand(x['zero_ratio'], x['cv']), 
    axis=1
)

print("📊 Классификация спроса:")
print(demand_df['demand_type'].value_counts())
print(f"\n📊 Распределение по типам:")
print(demand_df.groupby('demand_type').agg({
    'mean_demand': 'mean',
    'cv': 'mean',
    'zero_ratio': 'mean'
}).round(2))


In [ ]:
# Визуализация для примера товара
example_solo_code = all_solo_codes[0]
example_data = total_sales[total_sales['solo-code'] == example_solo_code].sort_values('Дата')

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# График продаж
axes[0].plot(example_data['Дата'], example_data['Количество упак.'], linewidth=1.5)
axes[0].set_title(f'Продажи по дням: {example_solo_code}', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Дата')
axes[0].set_ylabel('Количество упаковок')
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Распределение спроса
axes[1].hist(example_data['Количество упак.'], bins=30, edgecolor='black', alpha=0.7)
axes[1].set_title(f'Распределение спроса: {example_solo_code}', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Количество упаковок')
axes[1].set_ylabel('Частота')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Метрики для примера
example_metrics = demand_df[demand_df['solo-code'] == example_solo_code].iloc[0]
print(f"\n📊 Метрики для {example_solo_code}:")
print(f"  Тип спроса: {example_metrics['demand_type']}")
print(f"  Средний спрос: {example_metrics['mean_demand']:.2f}")
print(f"  CV: {example_metrics['cv']:.2f}")
print(f"  Доля нулей: {example_metrics['zero_ratio']:.2%}")


## 4. Генерация календарных признаков

Создаем признаки без внешних данных:
- Базовые признаки (день недели, месяц, квартал и т.д.)
- Циклические признаки (sin/cos)
- Праздники РФ с окнами влияния


In [ ]:
def create_calendar_features(df):
    """Создание календарных признаков"""
    df = df.copy()
    
    # Базовые признаки
    df['day_of_week'] = df['Дата'].dt.dayofweek  # 0=понедельник, 6=воскресенье
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['week_of_year'] = df['Дата'].dt.isocalendar().week
    df['month'] = df['Дата'].dt.month
    df['quarter'] = df['Дата'].dt.quarter
    df['year'] = df['Дата'].dt.year
    df['day_of_month'] = df['Дата'].dt.day
    df['day_of_year'] = df['Дата'].dt.dayofyear
    
    # Циклические признаки для месяца (12 месяцев)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    # Циклические признаки для недели (52 недели)
    df['week_sin'] = np.sin(2 * np.pi * df['week_of_year'] / 52)
    df['week_cos'] = np.cos(2 * np.pi * df['week_of_year'] / 52)
    
    # Циклические признаки для дня недели (7 дней)
    df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    return df

# Применяем к продажам
total_sales = create_calendar_features(total_sales)
wb_sales_processed = create_calendar_features(wb_sales_processed)
ozon_sales_processed = create_calendar_features(ozon_sales_processed)

print("✅ Базовые календарные признаки созданы")
print(f"\n📊 Пример признаков:")
print(total_sales[['Дата', 'day_of_week', 'is_weekend', 'month', 'quarter', 
                   'month_sin', 'month_cos']].head(10))


In [ ]:
def create_holiday_features(df):
    """Создание признаков праздников РФ с окнами влияния"""
    df = df.copy()
    
    # Новый год: 15.12 - 10.01
    df['is_new_year_period'] = 0
    df.loc[
        ((df['Дата'].dt.month == 12) & (df['Дата'].dt.day >= 15)) |
        ((df['Дата'].dt.month == 1) & (df['Дата'].dt.day <= 10)),
        'is_new_year_period'
    ] = 1
    
    # 14 февраля: ±7 дней
    df['is_feb_14_window'] = 0
    for year in df['Дата'].dt.year.unique():
        feb_14 = pd.Timestamp(f'{year}-02-14')
        mask = (df['Дата'] >= feb_14 - timedelta(days=7)) & (df['Дата'] <= feb_14 + timedelta(days=7))
        df.loc[mask, 'is_feb_14_window'] = 1
    
    # 23 февраля: ±7 дней
    df['is_feb_23_window'] = 0
    for year in df['Дата'].dt.year.unique():
        feb_23 = pd.Timestamp(f'{year}-02-23')
        mask = (df['Дата'] >= feb_23 - timedelta(days=7)) & (df['Дата'] <= feb_23 + timedelta(days=7))
        df.loc[mask, 'is_feb_23_window'] = 1
    
    # 8 марта: ±7 дней
    df['is_mar_8_window'] = 0
    for year in df['Дата'].dt.year.unique():
        mar_8 = pd.Timestamp(f'{year}-03-08')
        mask = (df['Дата'] >= mar_8 - timedelta(days=7)) & (df['Дата'] <= mar_8 + timedelta(days=7))
        df.loc[mask, 'is_mar_8_window'] = 1
    
    # 1 сентября: ±7 дней
    df['is_sep_1_window'] = 0
    for year in df['Дата'].dt.year.unique():
        sep_1 = pd.Timestamp(f'{year}-09-01')
        mask = (df['Дата'] >= sep_1 - timedelta(days=7)) & (df['Дата'] <= sep_1 + timedelta(days=7))
        df.loc[mask, 'is_sep_1_window'] = 1
    
    # Черная пятница: последние 2 недели ноября
    df['is_black_friday_period'] = 0
    df.loc[
        (df['Дата'].dt.month == 11) & (df['Дата'].dt.day >= 15),
        'is_black_friday_period'
    ] = 1
    
    return df

# Применяем праздничные признаки
total_sales = create_holiday_features(total_sales)
wb_sales_processed = create_holiday_features(wb_sales_processed)
ozon_sales_processed = create_holiday_features(ozon_sales_processed)

print("✅ Праздничные признаки созданы")
print(f"\n📊 Статистика по праздникам:")
holiday_cols = ['is_new_year_period', 'is_feb_14_window', 'is_feb_23_window', 
                'is_mar_8_window', 'is_sep_1_window', 'is_black_friday_period']
print(total_sales[holiday_cols].sum())


In [ ]:
# Корреляция праздников с продажами
holiday_corr = []
for solo_code in all_solo_codes[:10]:  # Первые 10 товаров
    data = total_sales[total_sales['solo-code'] == solo_code]
    for holiday in holiday_cols:
        if data[holiday].sum() > 0:  # Если есть праздничные дни
            corr = data['Количество упак.'].corr(data[holiday])
            holiday_corr.append({
                'solo-code': solo_code,
                'holiday': holiday,
                'correlation': corr
            })

if holiday_corr:
    holiday_corr_df = pd.DataFrame(holiday_corr)
    print("\n📊 Корреляция праздников с продажами (примеры):")
    print(holiday_corr_df.groupby('holiday')['correlation'].mean().sort_values(ascending=False))

# Пример календарных признаков
print("\n📊 Пример календарных признаков:")
example_calendar = total_sales[
    ['Дата', 'day_of_week', 'is_weekend', 'month', 'is_new_year_period', 
     'is_feb_14_window', 'is_mar_8_window', 'is_black_friday_period']
].head(20)
print(example_calendar)


## 5. Обработка бизнес-ограничений

### 5.1. Товары на вывод (withdraw)
### 5.2. Дефектура (defecture)


In [ ]:
def apply_withdraw_constraint(sales_df, stocks_wb, stocks_ozon, stocks_our, withdraw_df):
    """Применение ограничений для товаров на вывод"""
    if len(withdraw_df) == 0:
        return sales_df.copy(), stocks_wb.copy(), stocks_ozon.copy(), stocks_our.copy()
    
    withdraw_codes = set(withdraw_df['solo-code'].unique())
    
    sales_df = sales_df.copy()
    stocks_wb = stocks_wb.copy()
    stocks_ozon = stocks_ozon.copy()
    stocks_our = stocks_our.copy()
    
    # Для товаров на вывод: продажи = 0, остатки равномерно распределяются
    for solo_code in withdraw_codes:
        # Продажи = 0
        sales_df.loc[sales_df['solo-code'] == solo_code, 'Количество упак.'] = 0
        
        # Равномерное распределение остатков до нуля
        for stocks_df in [stocks_wb, stocks_ozon, stocks_our]:
            mask = stocks_df['solo-code'] == solo_code
            if mask.sum() > 0:
                total_stock = stocks_df.loc[mask, 'Остаток'].iloc[-1]  # Последний остаток
                days_left = len(stocks_df.loc[mask])
                if days_left > 0:
                    daily_decrease = total_stock / days_left
                    stocks_df.loc[mask, 'Остаток'] = np.maximum(
                        0, 
                        total_stock - np.arange(days_left) * daily_decrease
                    )
    
    return sales_df, stocks_wb, stocks_ozon, stocks_our

# Применяем ограничения на вывод
total_sales, wb_stocks_processed, ozon_stocks_processed, our_stocks_processed = apply_withdraw_constraint(
    total_sales, wb_stocks_processed, ozon_stocks_processed, our_stocks_processed, withdraw
)

print("✅ Ограничения на вывод применены")
if len(withdraw) > 0:
    print(f"📦 Товаров на вывод: {len(withdraw)}")
    print(f"📦 Коды: {withdraw['solo-code'].unique()[:5]}")
else:
    print("📦 Товаров на вывод нет")


In [ ]:
def apply_defecture_constraint(sales_df, defecture_df):
    """Применение ограничений дефектуры"""
    if len(defecture_df) == 0:
        return sales_df.copy()
    
    sales_df = sales_df.copy()
    
    for idx, row in defecture_df.iterrows():
        solo_code = row['solo-code']
        end_date = row['Дата окончания дефектуры']
        
        if pd.isna(end_date):
            continue
        
        # В период до окончания дефектуры: продажи = 0
        mask = (sales_df['solo-code'] == solo_code) & (sales_df['Дата'] <= end_date)
        sales_df.loc[mask, 'Количество упак.'] = 0
    
    return sales_df

# Применяем ограничения дефектуры
total_sales = apply_defecture_constraint(total_sales, defecture)

print("✅ Ограничения дефектуры применены")
if len(defecture) > 0:
    print(f"⚠️ Товаров в дефектуре: {len(defecture)}")
    print(f"⚠️ Примеры:")
    print(defecture.head())
else:
    print("✅ Дефектуры нет")


## 6. Модели прогнозирования

Реализуем набор моделей для каждого solo-code:
- Baseline (Naive, Moving Average, Seasonal Naive)
- Для intermittent demand (Croston, SBA, TSB)
- Продвинутые (Linear Regression, ARIMA, SARIMA, SARIMAX, Prophet)


In [ ]:
# Baseline модели

def naive_forecast(data, horizon):
    """Naive: последнее значение"""
    last_value = data.iloc[-1]
    return [last_value] * horizon

def moving_average_forecast(data, window=7, horizon=30):
    """Moving Average"""
    ma_value = data[-window:].mean()
    return [ma_value] * horizon

def seasonal_naive_forecast(data, season_length=7, horizon=30):
    """Seasonal Naive: значение с прошлого сезона"""
    if len(data) < season_length:
        return [data.mean()] * horizon
    seasonal_values = data[-season_length:].values
    forecast = []
    for i in range(horizon):
        forecast.append(seasonal_values[i % season_length])
    return forecast

print("✅ Baseline модели определены")


In [ ]:
# Модели для intermittent demand

def croston_forecast(data, alpha=0.1, horizon=30):
    """Croston's method для прерывистого спроса"""
    if len(data) < 2:
        return [data.mean()] * horizon
    
    # Разделяем на ненулевые значения и интервалы
    non_zero = data[data > 0]
    if len(non_zero) < 2:
        return [non_zero.mean() if len(non_zero) > 0 else 0] * horizon
    
    # Экспоненциальное сглаживание размера заказа
    sizes = non_zero.values
    avg_size = sizes[0]
    for size in sizes[1:]:
        avg_size = alpha * size + (1 - alpha) * avg_size
    
    # Экспоненциальное сглаживание интервала
    intervals = []
    last_idx = 0
    for i in range(1, len(data)):
        if data.iloc[i] > 0:
            intervals.append(i - last_idx)
            last_idx = i
    
    if len(intervals) < 2:
        avg_interval = len(data) / max(len(non_zero), 1)
    else:
        avg_interval = intervals[0]
        for interval in intervals[1:]:
            avg_interval = alpha * interval + (1 - alpha) * avg_interval
    
    # Прогноз = средний размер / средний интервал
    forecast_demand = avg_size / max(avg_interval, 1)
    
    return [forecast_demand] * horizon

def sba_forecast(data, alpha=0.1, horizon=30):
    """Syntetos-Boylan Approximation"""
    croston_fc = croston_forecast(data, alpha, horizon)
    # SBA = Croston * (1 - alpha/2)
    return [x * (1 - alpha / 2) for x in croston_fc]

def tsb_forecast(data, alpha=0.1, beta=0.1, horizon=30):
    """Teunter-Syntetos-Babai method"""
    if len(data) < 2:
        return [data.mean()] * horizon
    
    non_zero = data[data > 0]
    if len(non_zero) < 2:
        return [non_zero.mean() if len(non_zero) > 0 else 0] * horizon
    
    # Размер заказа
    sizes = non_zero.values
    avg_size = sizes[0]
    for size in sizes[1:]:
        avg_size = alpha * size + (1 - alpha) * avg_size
    
    # Вероятность ненулевого спроса
    prob = (data > 0).sum() / len(data)
    last_prob = prob
    for i in range(1, len(data)):
        if data.iloc[i] > 0:
            last_prob = beta * 1 + (1 - beta) * last_prob
        else:
            last_prob = beta * 0 + (1 - beta) * last_prob
    
    forecast_demand = avg_size * last_prob
    return [forecast_demand] * horizon

print("✅ Модели для intermittent demand определены")


In [ ]:
# Продвинутые модели

def linear_regression_forecast(train_data, test_features, calendar_features):
    """Linear Regression с лагами и календарными признаками"""
    if len(train_data) < 10:
        return [train_data.mean()] * len(test_features)
    
    # Создание лагов
    lags = [1, 7, 14, 30]
    X_train = []
    y_train = []
    
    for i in range(max(lags), len(train_data)):
        features = []
        # Лаги
        for lag in lags:
            if i - lag >= 0:
                features.append(train_data.iloc[i - lag])
            else:
                features.append(0)
        # Календарные признаки
        if calendar_features is not None and i < len(calendar_features):
            cal_features = calendar_features.iloc[i][
                ['day_of_week', 'is_weekend', 'month', 'quarter', 
                 'month_sin', 'month_cos', 'dow_sin', 'dow_cos'] +
                ['is_new_year_period', 'is_feb_14_window', 'is_feb_23_window', 
                 'is_mar_8_window', 'is_black_friday_period']
            ].values
            features.extend(cal_features)
        else:
            features.extend([0] * 13)
        
        X_train.append(features)
        y_train.append(train_data.iloc[i])
    
    if len(X_train) == 0:
        return [train_data.mean()] * len(test_features)
    
    # Обучение модели
    try:
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Прогноз
        forecast = []
        last_values = train_data[-max(lags):].values.tolist()
        
        for i, test_row in enumerate(test_features.iterrows()):
            features = []
            # Лаги из последних значений
            for lag in lags:
                if len(last_values) >= lag:
                    features.append(last_values[-lag])
                else:
                    features.append(0)
            # Календарные признаки
            cal_features = test_row[1][
                ['day_of_week', 'is_weekend', 'month', 'quarter',
                 'month_sin', 'month_cos', 'dow_sin', 'dow_cos'] +
                ['is_new_year_period', 'is_feb_14_window', 'is_feb_23_window',
                 'is_mar_8_window', 'is_black_friday_period']
            ].values
            features.extend(cal_features)
            
            pred = model.predict([features])[0]
            forecast.append(max(0, pred))  # Негативные значения не допускаем
            last_values.append(pred)
        return forecast
    except:
        return [train_data.mean()] * len(test_features)

print("✅ Linear Regression модель определена")


In [ ]:
def arima_forecast(data, horizon=30, order=(1, 1, 1)):
    """ARIMA модель"""
    if len(data) < 10:
        return [data.mean()] * horizon
    
    try:
        # Убираем нули для стабильности
        data_clean = data.copy()
        data_clean = data_clean.replace(0, 0.01)
        
        model = ARIMA(data_clean, order=order)
        fitted_model = model.fit()
        forecast = fitted_model.forecast(steps=horizon)
        return [max(0, x) for x in forecast]
    except:
        return [data.mean()] * horizon

def sarima_forecast(data, horizon=30, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7)):
    """SARIMA модель"""
    if len(data) < 20:
        return [data.mean()] * horizon
    
    try:
        data_clean = data.copy()
        data_clean = data_clean.replace(0, 0.01)
        
        model = SARIMAX(data_clean, order=order, seasonal_order=seasonal_order)
        fitted_model = model.fit(disp=False)
        forecast = fitted_model.forecast(steps=horizon)
        return [max(0, x) for x in forecast]
    except:
        return [data.mean()] * horizon

def sarimax_forecast(data, exog_train, exog_test, horizon=30):
    """SARIMAX с экзогенными переменными (календарь)"""
    if len(data) < 20 or exog_train is None or exog_test is None:
        return [data.mean()] * horizon
    
    try:
        data_clean = data.copy()
        data_clean = data_clean.replace(0, 0.01)
        
        # Выбираем ключевые календарные признаки
        exog_cols = ['is_weekend', 'month_sin', 'month_cos', 
                     'is_new_year_period', 'is_black_friday_period']
        exog_train_sel = exog_train[exog_cols] if all(c in exog_train.columns for c in exog_cols) else None
        exog_test_sel = exog_test[exog_cols] if all(c in exog_test.columns for c in exog_cols) else None
        
        if exog_train_sel is None or exog_test_sel is None:
            return sarima_forecast(data, horizon)
        
        model = SARIMAX(data_clean, exog=exog_train_sel, 
                       order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))
        fitted_model = model.fit(disp=False)
        forecast = fitted_model.forecast(steps=horizon, exog=exog_test_sel)
        return [max(0, x) for x in forecast]
    except:
        return [data.mean()] * horizon

print("✅ ARIMA/SARIMA/SARIMAX модели определены")


In [ ]:
def prophet_forecast(data, dates, horizon_dates, holidays_df=None):
    """Prophet модель с праздниками"""
    if len(data) < 30:
        return [data.mean()] * len(horizon_dates)
    
    try:
        # Подготовка данных для Prophet
        prophet_data = pd.DataFrame({
            'ds': dates,
            'y': data.values
        })
        
        # Праздники для Prophet
        if holidays_df is None:
            holidays_list = []
            # Новый год
            for year in [2024, 2025]:
                holidays_list.append({'holiday': 'new_year', 'ds': f'{year}-12-31', 'lower_window': -15, 'upper_window': 10})
                holidays_list.append({'holiday': 'feb_14', 'ds': f'{year}-02-14', 'lower_window': -7, 'upper_window': 7})
                holidays_list.append({'holiday': 'feb_23', 'ds': f'{year}-02-23', 'lower_window': -7, 'upper_window': 7})
                holidays_list.append({'holiday': 'mar_8', 'ds': f'{year}-03-08', 'lower_window': -7, 'upper_window': 7})
                holidays_list.append({'holiday': 'sep_1', 'ds': f'{year}-09-01', 'lower_window': -7, 'upper_window': 7})
            
            holidays_df = pd.DataFrame(holidays_list)
            holidays_df['ds'] = pd.to_datetime(holidays_df['ds'])
        
        # Обучение модели
        model = Prophet(holidays=holidays_df, yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)
        model.fit(prophet_data)
        
        # Прогноз
        future = pd.DataFrame({'ds': horizon_dates})
        forecast = model.predict(future)
        
        return [max(0, x) for x in forecast['yhat'].values]
    except Exception as e:
        return [data.mean()] * len(horizon_dates)

print("✅ Prophet модель определена")


## 7. Валидация и выбор модели

Rolling backtesting с метриками: MAE, RMSE, MAPE


In [ ]:
def calculate_metrics(y_true, y_pred):
    """Расчет метрик прогнозирования"""
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # MAE
    mae = mean_absolute_error(y_true, y_pred)
    
    # RMSE
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    # MAPE (устойчивый к нулям)
    mask = y_true > 0
    if mask.sum() > 0:
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        mape = 0
    
    return {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}

def rolling_backtest(data, calendar_data, models_dict, train_size=180, test_size=30, step=30):
    """Rolling backtesting для всех моделей"""
    results = []
    
    if len(data) < train_size + test_size:
        return pd.DataFrame()
    
    for start_test in range(train_size, len(data) - test_size + 1, step):
        train_data = data.iloc[:start_test]
        test_data = data.iloc[start_test:start_test + test_size]
        
        train_calendar = calendar_data.iloc[:start_test] if calendar_data is not None else None
        test_calendar = calendar_data.iloc[start_test:start_test + test_size] if calendar_data is not None else None
        
        for model_name, model_func in models_dict.items():
            try:
                if model_name == 'Linear Regression':
                    forecast = model_func(train_data, test_calendar, train_calendar)
                elif model_name == 'SARIMAX':
                    forecast = model_func(train_data, train_calendar, test_calendar, len(test_data))
                elif model_name == 'Prophet':
                    train_dates = train_calendar['Дата'] if train_calendar is not None else pd.date_range('2024-01-01', periods=len(train_data), freq='D')
                    test_dates = test_calendar['Дата'] if test_calendar is not None else pd.date_range('2024-01-01', periods=len(test_data), freq='D')
                    forecast = model_func(train_data, train_dates, test_dates)
                else:
                    forecast = model_func(train_data, len(test_data))
                
                metrics = calculate_metrics(test_data.values, forecast[:len(test_data)])
                metrics['model'] = model_name
                metrics['fold'] = start_test
                results.append(metrics)
            except Exception as e:
                continue
    
    return pd.DataFrame(results)

print("✅ Функции валидации определены")


In [ ]:
# Тестирование моделей на примере товара
example_solo_code = all_solo_codes[0]
example_sales = total_sales[total_sales['solo-code'] == example_solo_code].sort_values('Дата')
example_calendar = example_sales.copy()

# Подготовка данных
sales_series = example_sales['Количество упак.'].reset_index(drop=True)
demand_type = demand_df[demand_df['solo-code'] == example_solo_code]['demand_type'].iloc[0]

print(f"📦 Тестирование моделей для {example_solo_code}")
print(f"📊 Тип спроса: {demand_type}")
print(f"📊 Длина ряда: {len(sales_series)}")

# Выбор моделей в зависимости от типа спроса
models_to_test = {}

# Baseline всегда
models_to_test['Naive'] = lambda data, h: naive_forecast(data, h)
models_to_test['Moving Average'] = lambda data, h: moving_average_forecast(data, window=7, horizon=h)
models_to_test['Seasonal Naive'] = lambda data, h: seasonal_naive_forecast(data, season_length=7, horizon=h)

# Для intermittent
if demand_type == 'intermittent':
    models_to_test['Croston'] = lambda data, h: croston_forecast(data, horizon=h)
    models_to_test['SBA'] = lambda data, h: sba_forecast(data, horizon=h)
    models_to_test['TSB'] = lambda data, h: tsb_forecast(data, horizon=h)

# Продвинутые (если достаточно данных)
if len(sales_series) >= 30:
    models_to_test['Linear Regression'] = lambda data, cal, cal_train: linear_regression_forecast(data, cal, cal_train)
    models_to_test['ARIMA'] = lambda data, h: arima_forecast(data, horizon=h)
    
if len(sales_series) >= 60:
    models_to_test['SARIMA'] = lambda data, h: sarima_forecast(data, horizon=h)
    models_to_test['SARIMAX'] = lambda data, exog_train, exog_test, h: sarimax_forecast(data, exog_train, exog_test, h)
    
if len(sales_series) >= 90:
    models_to_test['Prophet'] = lambda data, dates, test_dates: prophet_forecast(data, dates, test_dates)

# Backtesting
if len(sales_series) >= 180:
    backtest_results = rolling_backtest(
        sales_series, 
        example_calendar,
        models_to_test,
        train_size=120,
        test_size=30,
        step=30
    )
    
    if len(backtest_results) > 0:
        print("\n📊 Результаты backtesting:")
        model_comparison = backtest_results.groupby('model').agg({
            'MAE': 'mean',
            'RMSE': 'mean',
            'MAPE': 'mean'
        }).round(2).sort_values('MAE')
        print(model_comparison)
        
        # Выбор лучшей модели
        best_model = model_comparison.index[0]
        print(f"\n🏆 Лучшая модель: {best_model}")
        print(f"   MAE: {model_comparison.loc[best_model, 'MAE']:.2f}")
        print(f"   RMSE: {model_comparison.loc[best_model, 'RMSE']:.2f}")
        print(f"   MAPE: {model_comparison.loc[best_model, 'MAPE']:.2f}%")
    else:
        print("⚠️ Недостаточно данных для backtesting")
else:
    print("⚠️ Недостаточно данных для полного backtesting (нужно минимум 180 дней)")


## 8. Rolling-прогноз продаж на 18 месяцев

Создаем прогноз для каждого solo-code на 18 месяцев вперед


In [ ]:
def generate_forecast(solo_code, sales_data, calendar_data, demand_type, models_dict, horizon_months=18):
    """Генерация прогноза для одного товара"""
    # Фильтрация данных
    item_data = sales_data[sales_data['solo-code'] == solo_code].sort_values('Дата')
    item_calendar = calendar_data[calendar_data['solo-code'] == solo_code].sort_values('Дата')
    
    if len(item_data) == 0:
        return None
    
    sales_series = item_data['Количество упак.'].reset_index(drop=True)
    
    # Создание календаря для прогноза
    last_date = item_data['Дата'].max()
    forecast_dates = pd.date_range(last_date + timedelta(days=1), periods=horizon_months * 30, freq='D')
    forecast_calendar = create_calendar_features(pd.DataFrame({'Дата': forecast_dates}))
    forecast_calendar = create_holiday_features(forecast_calendar)
    forecast_calendar['solo-code'] = solo_code
    
    # Выбор модели в зависимости от типа спроса и длины истории
    selected_model = None
    model_name = None
    
    # Логика выбора модели
    if demand_type == 'intermittent' and len(sales_series) >= 20:
        # Для прерывистого спроса
        if 'TSB' in models_dict:
            selected_model = lambda: tsb_forecast(sales_series, horizon=len(forecast_dates))
            model_name = 'TSB'
        elif 'Croston' in models_dict:
            selected_model = lambda: croston_forecast(sales_series, horizon=len(forecast_dates))
            model_name = 'Croston'
        else:
            selected_model = lambda: moving_average_forecast(sales_series, horizon=len(forecast_dates))
            model_name = 'Moving Average'
    elif len(sales_series) >= 90:
        # Для длинной истории - Prophet или SARIMA
        if 'Prophet' in models_dict:
            train_dates = item_calendar['Дата'].values
            selected_model = lambda: prophet_forecast(sales_series, train_dates, forecast_dates)
            model_name = 'Prophet'
        elif 'SARIMA' in models_dict:
            selected_model = lambda: sarima_forecast(sales_series, horizon=len(forecast_dates))
            model_name = 'SARIMA'
        else:
            selected_model = lambda: sarima_forecast(sales_series, horizon=len(forecast_dates))
            model_name = 'SARIMA'
    elif len(sales_series) >= 30:
        # Для средней истории - ARIMA или Linear Regression
        if 'ARIMA' in models_dict:
            selected_model = lambda: arima_forecast(sales_series, horizon=len(forecast_dates))
            model_name = 'ARIMA'
        elif 'Linear Regression' in models_dict:
            selected_model = lambda: linear_regression_forecast(sales_series, forecast_calendar, item_calendar)
            model_name = 'Linear Regression'
        else:
            selected_model = lambda: moving_average_forecast(sales_series, horizon=len(forecast_dates))
            model_name = 'Moving Average'
    else:
        # Для короткой истории - простые модели
        selected_model = lambda: moving_average_forecast(sales_series, horizon=len(forecast_dates))
        model_name = 'Moving Average'
    
    # Генерация прогноза
    try:
        forecast_values = selected_model()
        forecast_values = [max(0, x) for x in forecast_values]  # Негативные значения не допускаем
    except Exception as e:
        # Fallback к простой модели
        forecast_values = moving_average_forecast(sales_series, horizon=len(forecast_dates))
        model_name = 'Moving Average (fallback)'
    
    # Создание результата
    result = pd.DataFrame({
        'Дата': forecast_dates[:len(forecast_values)],
        'solo-code': solo_code,
        'Прогноз': forecast_values,
        'Модель': model_name
    })
    
    return result

print("✅ Функция генерации прогноза определена")


In [ ]:
# Генерация прогнозов для всех товаров
all_forecasts = []

print("📊 Генерация прогнозов для всех товаров...")
for idx, solo_code in enumerate(all_solo_codes):
    if (idx + 1) % 10 == 0:
        print(f"  Обработано: {idx + 1}/{len(all_solo_codes)}")
    
    demand_type = demand_df[demand_df['solo-code'] == solo_code]['demand_type'].iloc[0]
    
    # Прогноз для суммарных продаж
    forecast_total = generate_forecast(
        solo_code, 
        total_sales, 
        total_sales,
        demand_type,
        models_to_test,
        horizon_months=18
    )
    
    if forecast_total is not None:
        forecast_total['Канал'] = 'Total'
        all_forecasts.append(forecast_total)
    
    # Прогноз для WB
    forecast_wb = generate_forecast(
        solo_code,
        wb_sales_processed,
        wb_sales_processed,
        demand_type,
        models_to_test,
        horizon_months=18
    )
    
    if forecast_wb is not None:
        forecast_wb['Канал'] = 'WB'
        all_forecasts.append(forecast_wb)
    
    # Прогноз для Ozon
    forecast_ozon = generate_forecast(
        solo_code,
        ozon_sales_processed,
        ozon_sales_processed,
        demand_type,
        models_to_test,
        horizon_months=18
    )
    
    if forecast_ozon is not None:
        forecast_ozon['Канал'] = 'Ozon'
        all_forecasts.append(forecast_ozon)

if all_forecasts:
    forecasts_df = pd.concat(all_forecasts, ignore_index=True)
    print(f"\n✅ Прогнозы сгенерированы: {forecasts_df.shape}")
    print(f"📊 Уникальных товаров: {forecasts_df['solo-code'].nunique()}")
    print(f"📊 Каналов: {forecasts_df['Канал'].unique()}")
    print(f"\n📊 Пример прогноза:")
    print(forecasts_df.head(10))
else:
    print("⚠️ Прогнозы не сгенерированы")


In [ ]:
# Визуализация прогноза для примера товара
if 'forecasts_df' in locals() and len(forecasts_df) > 0:
    example_solo_code = all_solo_codes[0]
    
    # Фактические данные
    actual_total = total_sales[total_sales['solo-code'] == example_solo_code].sort_values('Дата')
    actual_wb = wb_sales_processed[wb_sales_processed['solo-code'] == example_solo_code].sort_values('Дата')
    actual_ozon = ozon_sales_processed[ozon_sales_processed['solo-code'] == example_solo_code].sort_values('Дата')
    
    # Прогнозы
    forecast_total = forecasts_df[(forecasts_df['solo-code'] == example_solo_code) & 
                                  (forecasts_df['Канал'] == 'Total')].sort_values('Дата')
    forecast_wb = forecasts_df[(forecasts_df['solo-code'] == example_solo_code) & 
                                (forecasts_df['Канал'] == 'WB')].sort_values('Дата')
    forecast_ozon = forecasts_df[(forecasts_df['solo-code'] == example_solo_code) & 
                                 (forecasts_df['Канал'] == 'Ozon')].sort_values('Дата')
    
    fig, axes = plt.subplots(3, 1, figsize=(16, 12))
    
    # Total
    axes[0].plot(actual_total['Дата'], actual_total['Количество упак.'], 
                 label='Факт', linewidth=2, color='blue')
    if len(forecast_total) > 0:
        axes[0].plot(forecast_total['Дата'], forecast_total['Прогноз'], 
                    label='Прогноз', linewidth=2, color='red', linestyle='--')
        axes[0].axvline(x=actual_total['Дата'].max(), color='gray', linestyle=':', alpha=0.7, label='Начало прогноза')
    axes[0].set_title(f'Прогноз продаж: {example_solo_code} - Total', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Дата')
    axes[0].set_ylabel('Количество упаковок')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # WB
    axes[1].plot(actual_wb['Дата'], actual_wb['Количество упак.'], 
                 label='Факт WB', linewidth=2, color='blue')
    if len(forecast_wb) > 0:
        axes[1].plot(forecast_wb['Дата'], forecast_wb['Прогноз'], 
                    label='Прогноз WB', linewidth=2, color='red', linestyle='--')
        axes[1].axvline(x=actual_wb['Дата'].max(), color='gray', linestyle=':', alpha=0.7)
    axes[1].set_title(f'Прогноз продаж: {example_solo_code} - Wildberries', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Дата')
    axes[1].set_ylabel('Количество упаковок')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Ozon
    axes[2].plot(actual_ozon['Дата'], actual_ozon['Количество упак.'], 
                 label='Факт Ozon', linewidth=2, color='blue')
    if len(forecast_ozon) > 0:
        axes[2].plot(forecast_ozon['Дата'], forecast_ozon['Прогноз'], 
                    label='Прогноз Ozon', linewidth=2, color='red', linestyle='--')
        axes[2].axvline(x=actual_ozon['Дата'].max(), color='gray', linestyle=':', alpha=0.7)
    axes[2].set_title(f'Прогноз продаж: {example_solo_code} - Ozon', fontsize=14, fontweight='bold')
    axes[2].set_xlabel('Дата')
    axes[2].set_ylabel('Количество упаковок')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Использованная модель: {forecast_total['Модель'].iloc[0] if len(forecast_total) > 0 else 'N/A'}")


## 9. Расчет страхового запаса

Формула: Safety Stock = Z * σ * sqrt(lead_time)

где:
- Z = 1.65 для service level 95%
- σ = стандартное отклонение дневного спроса
- lead_time = время поставки (по умолчанию 30 дней)


In [ ]:
def calculate_safety_stock(sales_data, service_level=0.95, lead_time=30):
    """Расчет страхового запаса для всех товаров"""
    # Z-score для service level
    from scipy import stats
    z_score = stats.norm.ppf(service_level)
    
    safety_stocks = []
    
    for solo_code in all_solo_codes:
        item_data = sales_data[sales_data['solo-code'] == solo_code].sort_values('Дата')
        
        if len(item_data) == 0:
            continue
        
        demand = item_data['Количество упак.'].values
        
        # Средний дневной спрос
        mean_demand = np.mean(demand)
        
        # Стандартное отклонение дневного спроса
        std_demand = np.std(demand)
        
        # Страховой запас
        safety_stock = z_score * std_demand * np.sqrt(lead_time)
        safety_stock = max(0, safety_stock)  # Не может быть отрицательным
        
        safety_stocks.append({
            'solo-code': solo_code,
            'mean_demand': mean_demand,
            'std_demand': std_demand,
            'safety_stock': safety_stock,
            'lead_time': lead_time,
            'service_level': service_level
        })
    
    return pd.DataFrame(safety_stocks)

# Расчет страхового запаса
safety_stock_df = calculate_safety_stock(total_sales, service_level=0.95, lead_time=30)

print("📊 Страховой запас рассчитан:")
print(safety_stock_df.head(10))
print(f"\n📊 Общая статистика:")
print(safety_stock_df[['mean_demand', 'std_demand', 'safety_stock']].describe())


## 10. Планирование отгрузок

На основе:
- Прогноза продаж
- Остатков на складах (WB, Ozon, наш склад)
- Страхового запаса
- Количества упаковок в коробе (count_box)

Рассчитываем:
- Чистую потребность
- Отгрузку в упаковках и коробах


In [ ]:
def plan_shipments(forecasts_df, stocks_wb, stocks_ozon, stocks_our, safety_stock_df, count_box_df, horizon_days=30):
    """Планирование отгрузок"""
    shipment_plan = []
    
    # Создание словаря count_box
    count_box_dict = dict(zip(count_box_df['solo-code'], count_box_df['Кол-во']))
    
    # Получаем последние остатки
    last_date = max(
        stocks_wb['Дата'].max(),
        stocks_ozon['Дата'].max(),
        stocks_our['Дата'].max()
    )
    
    last_stocks_wb = stocks_wb[stocks_wb['Дата'] == last_date].set_index('solo-code')['Остаток'].to_dict()
    last_stocks_ozon = stocks_ozon[stocks_ozon['Дата'] == last_date].set_index('solo-code')['Остаток'].to_dict()
    last_stocks_our = stocks_our[stocks_our['Дата'] == last_date].set_index('solo-code')['Остаток'].to_dict()
    
    # Словарь страхового запаса
    safety_stock_dict = dict(zip(safety_stock_df['solo-code'], safety_stock_df['safety_stock']))
    
    # Обработка прогнозов по каналам
    for channel in ['WB', 'Ozon']:
        channel_forecasts = forecasts_df[
            (forecasts_df['Канал'] == channel) & 
            (forecasts_df['Дата'] <= last_date + timedelta(days=horizon_days))
        ].copy()
        
        for solo_code in channel_forecasts['solo-code'].unique():
            item_forecast = channel_forecasts[channel_forecasts['solo-code'] == solo_code].sort_values('Дата')
            
            # Текущие остатки
            stock_wb = last_stocks_wb.get(solo_code, 0)
            stock_ozon = last_stocks_ozon.get(solo_code, 0)
            stock_our = last_stocks_our.get(solo_code, 0)
            
            # Страховой запас
            safety_stock = safety_stock_dict.get(solo_code, 0)
            
            # Количество в коробе
            boxes_per_pack = count_box_dict.get(solo_code, 1)
            
            # Прогноз продаж на период
            forecast_demand = item_forecast['Прогноз'].sum()
            
            # Текущий остаток на канале
            if channel == 'WB':
                current_stock = stock_wb
            else:
                current_stock = stock_ozon
            
            # Чистая потребность = прогноз + страховой запас - текущий остаток
            net_requirement = forecast_demand + safety_stock - current_stock
            net_requirement = max(0, net_requirement)  # Не может быть отрицательным
            
            # Округление до целого короба
            shipment_boxes = np.ceil(net_requirement / boxes_per_pack) if boxes_per_pack > 0 else 0
            shipment_packs = shipment_boxes * boxes_per_pack
            
            shipment_plan.append({
                'Дата_планирования': last_date,
                'solo-code': solo_code,
                'Канал': channel,
                'Прогноз_продаж': forecast_demand,
                'Остаток_WB': stock_wb,
                'Остаток_Ozon': stock_ozon,
                'Остаток_наш': stock_our,
                'Страховой_запас': safety_stock,
                'Чистая_потребность': net_requirement,
                'Отгрузка_упаковок': shipment_packs,
                'Отгрузка_коробов': shipment_boxes,
                'Упаковок_в_коробе': boxes_per_pack
            })
    
    return pd.DataFrame(shipment_plan)

# Планирование отгрузок
if 'forecasts_df' in locals() and len(forecasts_df) > 0:
    shipment_plan_df = plan_shipments(
        forecasts_df,
        wb_stocks_processed,
        ozon_stocks_processed,
        our_stocks_processed,
        safety_stock_df,
        count_box,
        horizon_days=30
    )
    
    print("📦 План отгрузок создан:")
    print(shipment_plan_df.head(20))
    print(f"\n📊 Сводка по отгрузкам:")
    print(shipment_plan_df.groupby('Канал').agg({
        'Отгрузка_упаковок': 'sum',
        'Отгрузка_коробов': 'sum'
    }).round(2))
else:
    print("⚠️ Прогнозы не найдены, план отгрузок не создан")


In [ ]:
print("=" * 80)
print("📊 ИТОГОВАЯ СВОДКА СИСТЕМЫ ПРОГНОЗИРОВАНИЯ")
print("=" * 80)

print(f"\n📦 Обработано товаров: {len(all_solo_codes)}")
print(f"📅 Период данных: {min_date.date()} - {max_date.date()}")
print(f"📅 Всего дней в истории: {(max_date - min_date).days + 1}")

print(f"\n📊 Классификация спроса:")
if 'demand_df' in locals():
    print(demand_df['demand_type'].value_counts())

print(f"\n📈 Использованные модели:")
if 'forecasts_df' in locals():
    model_usage = forecasts_df.groupby('Модель').size().sort_values(ascending=False)
    print(model_usage)

print(f"\n📦 Планирование отгрузок:")
if 'shipment_plan_df' in locals():
    total_shipment = shipment_plan_df.groupby('Канал').agg({
        'Отгрузка_упаковок': 'sum',
        'Отгрузка_коробов': 'sum',
        'solo-code': 'nunique'
    })
    print(total_shipment)

print(f"\n🏬 Страховой запас:")
if 'safety_stock_df' in locals():
    print(f"  Средний страховой запас: {safety_stock_df['safety_stock'].mean():.2f} упаковок")
    print(f"  Медианный страховой запас: {safety_stock_df['safety_stock'].median():.2f} упаковок")
    print(f"  Максимальный страховой запас: {safety_stock_df['safety_stock'].max():.2f} упаковок")

print("\n✅ Система прогнозирования успешно построена!")


In [ ]:
print("=" * 80)
print("⚠️ РИСКИ И РЕКОМЕНДАЦИИ")
print("=" * 80)

print("\n🔴 РИСКИ:")
print("1. Качество прогноза зависит от длины исторических данных")
print("   - Для товаров с короткой историей (<30 дней) используются простые модели")
print("   - Рекомендация: накапливать данные минимум 3-6 месяцев")
print("\n2. Прерывистый спрос (intermittent) сложен для прогнозирования")
print("   - Высокая доля нулевых дней снижает точность")
print("   - Рекомендация: использовать специализированные модели (Croston, TSB)")
print("\n3. Внешние факторы не учтены")
print("   - Промоакции, изменения цен, конкурентная среда")
print("   - Рекомендация: добавить внешние данные при наличии")
print("\n4. Дефектура и вывод товаров")
print("   - Могут существенно влиять на прогноз")
print("   - Рекомендация: регулярно обновлять справочники")
print("\n5. Сезонность")
print("   - Модели учитывают базовую сезонность, но могут не улавливать специфику категории")
print("   - Рекомендация: настраивать сезонность под категорию товара")

print("\n🟢 РЕКОМЕНДАЦИИ ПО РАЗВИТИЮ:")
print("1. Автоматизация обновления данных")
print("   - Ежедневная загрузка новых данных")
print("   - Автоматический пересчет прогнозов")
print("\n2. Мониторинг качества прогноза")
print("   - Отслеживание метрик (MAE, RMSE, MAPE) в реальном времени")
print("   - Алерты при значительном отклонении")
print("\n3. A/B тестирование моделей")
print("   - Сравнение разных моделей на новых данных")
print("   - Автоматический выбор лучшей модели")
print("\n4. Интеграция с внешними данными")
print("   - Промоакции, изменения цен")
print("   - Экономические индикаторы")
print("   - Данные конкурентов (если доступны)")
print("\n5. Мультиуровневое прогнозирование")
print("   - Прогноз на уровне категории + корректировка на уровне SKU")
print("   - Учет взаимосвязей между товарами")
print("\n6. Оптимизация запасов")
print("   - Учет ограничений по складам")
print("   - Минимизация общих затрат (хранение + дефицит)")
print("\n7. Визуализация и дашборды")
print("   - Интерактивные графики прогнозов")
print("   - Анализ отклонений")
print("   - Планы отгрузок в удобном формате")

print("\n✅ Система готова к использованию!")
